In [2]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
# from skimage.io import imread,imshow,show
from glob import glob
from os import listdir
import csv
# %matplotlib nbagg
# %matplotlib qt
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.draw import polygon_perimeter
from IPython.core.debugger import Tracer
import scipy.misc

import sys
print "This is the name of the script: ", sys.argv[0]
print "Number of arguments: ", len(sys.argv)
print "The arguments are: " , str(sys.argv)


start = 0;
stop = 10



working_path = "/work/vsankar/Project-Luna/segmentedNodules2/"
images_path = "/work/vsankar/Project-Luna/Output_toolbox1/images/"
saving_path = "/work/vsankar/Project-Luna/data/array/temp/"

def returnWindow(x,y,step):

    if x - step > 0 and x + step < 512:
        xl = x-step ; 
        xh = x+step-1 ; 
    elif x - step <= 0:
        xl = 1;
        xh = 96;
    elif  x + step >= size(1):   
        xl = size(1)-step*2 ;
        xh = size(1) - 1;
    
    
    if y - step > 0 and y + step < 512:
        yl = y-step ; 
        yh = y+step-1 ; 
    elif y - step <= 0:
        yl = 1;
        yh = 96;
    elif  y + step >= size(2):   
        yl = size(2)-step*2 ;
        yh = size(2) - 1;    
    return xl,xh,yl,yh



def returnSquareMask(msk,img):
#     try:
#     Tracer()()
    idxr,idxc = np.nonzero(msk)
    rmax = max(idxr)
    cmax = max(idxc)
    rmin = min(idxr)
    cmin = min(idxc)

    img1 = img.copy()

    img1_RGB = np.asarray(scipy.misc.toimage(img1,mode='L').convert("RGB"))
    img1_RGB.setflags(write=1)

    img1_RGB[rmin:rmax,cmin,0] = 255;
    img1_RGB[rmin:rmax,cmax,0] = 255;
    img1_RGB[rmin,cmin:cmax,0] = 255;
    img1_RGB[rmax,cmin:cmax,0] = 255;

    step=48;
    x = int(np.mean(idxr))
    y = int(np.mean(idxc))

    [xl,xh,yl,yh] = returnWindow(x,y,step);
    cropped_img = img1_RGB[xl:xh+1,yl:yh+1];

    smsk = np.zeros_like(msk);
    smsk[rmin:rmax,cmin:cmax] = 1
#     Tracer()()
    if len(cropped_img) != 96:
        Tracer()()
    
        # ploting
#     plt.subplot(1,2,1)
#     plt.imshow(msk)
#     plt.subplot(1,2,2)
#     plt.imshow(smsk)
#     plt.show()
#                 plt.subplot(1,3,3)
#     plt.imshow(img1)
#     plt.show()
#     Tracer()()
    
    
#     Tracer()()
    return smsk,cropped_img


# file_list=glob(working_path+"images_*.npy")

images = np.ndarray([1,1,96,96],dtype=np.float32)
original_mask = np.ndarray([1,1,512,512],dtype=np.float32)
square_mask = np.ndarray([1,1,512,512],dtype=np.float32)

mal = np.array([],dtype=np.float32)

slice_images = np.ndarray([1,1,512,512],dtype=np.float32)
nodule_images_border = np.ndarray([1,1,96,96,3],dtype=np.float32)

dir = listdir(working_path)
# Tracer()()
for pn in range(start,stop):
    print(pn)
    patientID = dir[pn]
    PatientID_path =  working_path + patientID + '/'
    slice_list =glob(PatientID_path+"slice*")

#     Tracer()() 
    slice_mal = list([]);
    try:
#         Tracer()()
        
        if len(slice_list)==0:
#             Tracer()()
            continue
        with open(PatientID_path+'malignancy.csv', 'rb') as f:
            reader = csv.reader(f)
            for row in reader:
                slice_mag_values = map(int,row)
                 
                slice_mal.append(np.mean([x for x in slice_mag_values if x]))
#             Tracer()() 

        for sl in range(len(slice_list)): 
#             Tracer()() 
            slice_path = slice_list[sl]

            try:
                sl_num = int(slice_list[sl][-2:])
            except:
                sl_num = int(slice_list[sl][-1])

            imagefiles = glob(slice_path+"/nodule_*")
            for n1 in range(len(imagefiles)):
                
#                 Tracer()() 
                rad_num = imagefiles[n1][-6]
                #nodule image
                img = np.ndarray([1,1,96,96],dtype=np.float32)
                image_name = slice_path + "/nodule_image"+str(rad_num)+".tiff"
                img[0,0] = plt.imread(image_name)
                images = np.append(images, img, axis=0)
                
#                 Tracer()() 
                #slice image
                slice_img = np.ndarray([1,1,512,512],dtype=np.float32)
                slice_image_name = images_path + patientID + '/slice'+ str(sl_num) + '.tif'
                slice_img[0,0] = plt.imread(slice_image_name)
                slice_images = np.append(slice_images, slice_img, axis=0)
                
#                 Tracer()() 
                #mask original
                msk = np.ndarray([1,1,512,512],dtype=np.float32)
                msk_name = slice_path + "/nodulemask"+str(rad_num)+".tiff"
                msk[0,0] = plt.imread(msk_name)
                original_mask = np.append(original_mask, msk, axis=0)
                
#                 Tracer()() 
                #mask square
                smsk = np.ndarray([1,1,512,512],dtype=np.float32)
                nodule_image_b = np.ndarray([1,1,96,96,3],dtype=np.float32)
                smsk[0,0],nodule_image_b[0,0] = returnSquareMask(msk[0,0],slice_img[0,0])
                
                square_mask = np.append(square_mask, msk, axis=0)
                nodule_images_border = np.append(nodule_images_border, nodule_image_b, axis=0)
                scipy.misc.imsave('/work/vsankar/Project-Luna/data/array/images/nodule_'+ str(pn) +'_'+ str(sl) +'_'+ str(n1) +'_'+'_.png', img[0,0])
                scipy.misc.imsave('/work/vsankar/Project-Luna/data/array/images/nodule_images_border_'+ str(pn) +'_'+ str(sl) +'_'+ str(n1) +'_'+'_.png', nodule_image_b[0,0])
                
#                 plt.subplot(1,2,1)
#                 plt.imshow(img[0,0])
#                 plt.subplot(1,2,2)
#                 plt.imshow(nodule_image_b[0,0])
#                 plt.show()
#                 Tracer()()
                #malignancy
                mal = np.append(mal,slice_mal[sl_num-1])
                

                
                
    except IOError as e:
        print e
#         Tracer()()
#     print mal.shape
#     print images.shape
    

    
    
    
images = np.delete(images, 0, 0)
nodule_images_border = np.delete(nodule_images_border, 0, 0)

    
np.save(saving_path+"Images_mask_"+str(start)+"_"+ str(stop)+"_.npy",images)
# np.save(saving_path+"square_mask_"+str(start)+"_"+ str(stop)+"_.npy",square_mask)
# np.save(saving_path+"originak_mask_"+str(start)+"_"+ str(stop)+"_.npy",original_mask)
# np.save(saving_path+"slice_images.npy",slice_images)
np.save(saving_path+"slice_images_border_"+str(start)+"_"+ str(stop)+"_.npy",nodule_images_border)
np.save(saving_path+"mal_mask_"+str(start)+"_"+ str(stop)+"_.npy",mal)

This is the name of the script:  /home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py
Number of arguments:  3
The arguments are:  ['/home/vsankar/pyenvs/LungCancer/lib/python2.7/site-packages/ipykernel/__main__.py', '-f', '/home/vsankar/.local/share/jupyter/runtime/kernel-cd12f414-cb9a-4703-934d-23f5f7fb765a.json']
0
1
2
3
4
5
6
7
8
9


In [ ]:
import numpy as np
from skimage import morphology
from skimage import measure
from sklearn.cluster import KMeans
from skimage.transform import resize
# from skimage.io import imread,imshow,show
from glob import glob
from os import listdir
import csv
# %matplotlib nbagg
# %matplotlib qt
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.draw import polygon_perimeter
from IPython.core.debugger import Tracer
import scipy.misc

working_path = "/work/vsankar/Project-Luna/segmentedNodules2/"
images_path = "/work/vsankar/Project-Luna/Output_toolbox1/images/"
saving_path = "/work/vsankar/Project-Luna/data/array/"

start = 1000
stop = 1010
imgaes = np.load(saving_path+"Images_mask_"+str(start)+"_"+ str(stop)+"_.npy")

square_mask = np.load(saving_path+"square_mask_"+str(start)+"_"+ str(stop)+"_.npy")
square_mask = np.delete(square_mask, 0, 0)



original_mask = np.load(saving_path+"originak_mask_"+str(start)+"_"+ str(stop)+"_.npy",)
original_mask = np.delete(original_mask, 0, 0)

slice_images_border = np.load(saving_path+"slice_images_border_"+str(start)+"_"+ str(stop)+"_.npy")

for j in range(len(slice_images_border)):
    scipy.misc.imsave('/work/vsankar/Project-Luna/data/array/images/slice_images_border_'+ str(j)+'_.png', slice_images_border[j,0])
    
    
mal = np.load(saving_path+"mal_mask_"+str(start)+"_"+ str(stop)+"_.npy")

# Tracer()()

# for i in range(1,10):
#     print(i)
#     start = i*100
#     stop = (i+1)*100
    
#     imgaes1 = np.load(saving_path+"Images_mask_"+str(start)+"_"+ str(stop)+"_.npy")
#     imgaes = np.append(imgaes, imgaes1, axis=0)

#     slice_images_border1 = np.load(saving_path+"slice_images_border_"+str(start)+"_"+ str(stop)+"_.npy")
#     slice_images_border = np.append(slice_images_border, slice_images_border1, axis=0)
    
#     mal1 = np.load(saving_path+"mal_mask_"+str(start)+"_"+ str(stop)+"_.npy")
#     mal = np.append(mal, mal1, axis=0)
    
# #     Tracer()()
    


In [ ]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.imshow(slice_images_border[0,0],cmap='gray')
plt.show()
plt.savefig('/work/vsankar/Project-Luna/data/array/myfig.png')

In [ ]:

scipy.misc.imsave('/work/vsankar/Project-Luna/data/array/outfile.jpg', slice_images_border[0,0])

In [ ]:
len(slice_images_border)


In [ ]:
images= np.delete(images, 0, 0)